# जनावर विशेषज्ञ प्रणाली कार्यान्वयन

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) बाट लिएको एक उदाहरण।

यस नमूनामा, हामी केही भौतिक विशेषताहरूको आधारमा जनावर पहिचान गर्नको लागि एक साधारण ज्ञान-आधारित प्रणाली कार्यान्वयन गर्नेछौं। प्रणालीलाई निम्न AND-OR रूखद्वारा प्रतिनिधित्व गर्न सकिन्छ (यो सम्पूर्ण रूखको एक भाग हो, हामी सजिलै थप नियमहरू थप्न सक्छौं):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.ne.png)


## हाम्रो आफ्नै विशेषज्ञ प्रणाली शेल ब्याकवर्ड इन्फरेन्सको साथ

उत्पादन नियमहरूमा आधारित ज्ञान प्रतिनिधित्वको लागि एउटा साधारण भाषा परिभाषित गर्ने प्रयास गरौं। हामी नियमहरू परिभाषित गर्न Python कक्षाहरूलाई कुञ्जीशब्दको रूपमा प्रयोग गर्नेछौं। मुख्यतया ३ प्रकारका कक्षाहरू हुनेछन्:
* `Ask` एउटा प्रश्नलाई प्रतिनिधित्व गर्छ जुन प्रयोगकर्तालाई सोध्न आवश्यक छ। यसमा सम्भावित उत्तरहरूको सेट समावेश हुन्छ।
* `If` एउटा नियमलाई प्रतिनिधित्व गर्छ, र यो नियमको सामग्री भण्डारण गर्नको लागि मात्र एक प्रकारको सिंट्याक्टिक सुगर हो।
* `AND`/`OR` कक्षाहरू रूखको AND/OR शाखाहरूलाई प्रतिनिधित्व गर्नका लागि हुन्। यीले भित्रका तर्कहरूको सूची मात्र भण्डारण गर्छन्। कोडलाई सरल बनाउन, सबै कार्यक्षमता अभिभावक कक्षा `Content` मा परिभाषित गरिएको छ।


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

हाम्रो प्रणालीमा, कार्य स्मृतिमा **तथ्यहरू** को सूची **गुण-मान जोडीहरू** को रूपमा समावेश हुनेछ। ज्ञान आधारलाई एउटा ठूलो शब्दकोशको रूपमा परिभाषित गर्न सकिन्छ जसले क्रियाहरू (नयाँ तथ्यहरू जुन कार्य स्मृतिमा समावेश गर्नुपर्छ) लाई सर्तहरूसँग नक्सा गर्दछ, जुन AND-OR अभिव्यक्तिहरूको रूपमा व्यक्त गरिन्छ। साथै, केही तथ्यहरूलाई `सोध्न` सकिन्छ।


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

पछाडि तर्फको अनुमान गर्नको लागि, हामी `Knowledgebase` नामक कक्षा परिभाषित गर्नेछौं। यसले समावेश गर्नेछ:
* कार्यरत `memory` - एउटा शब्दकोश जसले विशेषताहरूलाई मानहरूसँग नक्सा बनाउँछ
* Knowledgebase `rules` - माथि परिभाषित गरिएको ढाँचामा

दुई मुख्य विधिहरू छन्:
* `get` - कुनै विशेषताको मान प्राप्त गर्न प्रयोग गरिन्छ, आवश्यक परे अनुमान गर्दै। उदाहरणका लागि, `get('color')` ले रंग स्लटको मान प्राप्त गर्नेछ (आवश्यक परे सोध्नेछ, र पछि प्रयोगको लागि कार्यरत मेमोरीमा मान भण्डारण गर्नेछ)। यदि हामी `get('color:blue')` सोध्छौं भने, यो रंगको लागि सोध्नेछ, र त्यसपछि रंगको आधारमा `y`/`n` मान फर्काउनेछ।
* `eval` - वास्तविक अनुमान प्रदर्शन गर्ने विधि हो, अर्थात् AND/OR ट्रीको यात्रा गर्ने, उप-लक्ष्यहरूको मूल्यांकन गर्ने, आदि।


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

अब हामी हाम्रो जनावर ज्ञानको आधार परिभाषित गरौं र परामर्श सञ्चालन गरौं। ध्यान दिनुहोस् कि यो कलले तपाईंलाई प्रश्नहरू सोध्नेछ। तपाईं `y`/`n` टाइप गरेर हो-होइन प्रश्नहरूको उत्तर दिन सक्नुहुन्छ, वा लामो बहुविकल्पीय उत्तरहरू भएका प्रश्नहरूको लागि संख्या (0..N) निर्दिष्ट गरेर उत्तर दिन सक्नुहुन्छ।


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## PyKnow प्रयोग गरेर फर्वार्ड इन्फरेन्स

अर्को उदाहरणमा, हामी ज्ञान प्रतिनिधित्वका लागि उपलब्ध पुस्तकालयहरूमध्ये एक, [PyKnow](https://github.com/buguroo/pyknow/) प्रयोग गरेर फर्वार्ड इन्फरेन्स कार्यान्वयन गर्ने प्रयास गर्नेछौं। **PyKnow** एउटा पुस्तकालय हो जसले Python मा फर्वार्ड इन्फरेन्स प्रणालीहरू बनाउनका लागि प्रयोग गरिन्छ, जुन पुरानो क्लासिकल प्रणाली [CLIPS](http://www.clipsrules.net/index.html) जस्तै डिजाइन गरिएको छ।

हामीले फर्वार्ड चेइनिङ आफैं पनि धेरै समस्याबिना कार्यान्वयन गर्न सक्थ्यौं, तर साधारण कार्यान्वयनहरू प्रायः धेरै प्रभावकारी हुँदैनन्। नियम मिलानलाई अझ प्रभावकारी बनाउनका लागि एउटा विशेष एल्गोरिदम [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) प्रयोग गरिन्छ।


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

हामी हाम्रो प्रणालीलाई `KnowledgeEngine` को उपवर्गको रूपमा कक्षा रूपमा परिभाषित गर्नेछौं। प्रत्येक नियम `@Rule` एनोटेशनसहितको छुट्टै कार्यद्वारा परिभाषित गरिन्छ, जसले नियम कहिले कार्यान्वयन हुनुपर्छ भनेर निर्दिष्ट गर्दछ। नियमभित्र, हामी `declare` कार्य प्रयोग गरेर नयाँ तथ्यहरू थप्न सक्छौं, र ती तथ्यहरू थप्दा अगाडि निष्कर्ष इन्जिनद्वारा केही थप नियमहरू बोलाइनेछ।


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

एक पटक हामीले ज्ञान आधार परिभाषित गरेपछि, हामी हाम्रो कार्यरत स्मृतिलाई केही प्रारम्भिक तथ्यहरूद्वारा भरिन्छौं, र त्यसपछि `run()` विधिलाई कल गरेर निष्कर्ष निकाल्ने कार्य गर्दछौं। परिणामस्वरूप, तपाईंले देख्न सक्नुहुन्छ कि नयाँ निष्कर्ष निकालिएका तथ्यहरू कार्यरत स्मृतिमा थपिएका छन्, जसमा जनावरको अन्तिम तथ्य पनि समावेश छ (यदि हामीले सबै प्रारम्भिक तथ्यहरू सही रूपमा सेटअप गरेका छौं भने)।


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।  
